# Linear Regression

## Importing Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Generating Data

We begin by generating a test dataset by inserting gaussian noise to a cubic function $f(x) = ax^{3} + bx^{2} + cx + d$. The constants $a$, $b$, $c$ and $d$ will be chosen arbitrarily and the outputted dataset will be saved for future model learning.   

In [ ]:
def f(x):
    return 0.7 * (x**3) - 7 * (x**2) + 1 * (x) + 5

In [ ]:
X = np.linspace(-5, 11, 1000)
Y = f(X) + 20 * np.random.randn(X.shape[0])

In [ ]:
plt.scatter(X, Y, s=2)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('cubic function with noise and arbitrary constants')
plt.grid(True)

## Creating Linear Model

We now define a linear model that we will train with the generated data in order to *learn* the arbitrarily chosen parameters in the previous step.

Our model is a function of $x$, parameterized by weights $a_0,\, a_1,\, a_2,\, a_3$ and will be defined as follows:

$$
\begin{equation}
    f^*(x; a_0,\, a_1,\, a_2,\, a_3) = a_0x^3 + a_1x^2 + a_2x + a_3
\end{equation}
$$

For coding convenience, we store the constants in a **numpy array:** `a = [a0, a1, a2, a3]`.

In [ ]:
a = np.zeros(4)

def f_star(x):
    return a[0]*(x**3) + a[1]*(x**2) + a[2]*x + a[3]

The *loss function* which finds the error between our linear model's prediction $f^*(x)$ and the ground truth value $f(x)$ is optimizing the parameters and is thus a function of $\mathbf{a}$ and parametersized by $x$:

$$
\begin{equation}
    L(\mathbf{a};x) = \frac{1}{2n}\sum (f(x) - f^*(x;\mathbf{a}))^2 \tag{2}
\end{equation}
$$

In [ ]:
def Loss(X, Y):
    return (1/2) * np.mean((Y - f_star(X))**2)

## Stochastic Gradient Descent

The *gradient* of this loss function $w.r.t$ our unkonwn weights $\mathbf{a} = [a_0, \, a_1, \, a_2, \, a_3]$ is calculated:

$$
\begin{align*}
    \frac{d}{d\mathbf{a}} L(\mathbf{a}; x) &= \frac{d}{d\mathbf{a}} \left[ \frac{1}{2n}\sum (f(x) - f^*(x;\mathbf{a}))^2 \right] \\ \\
                      &= \frac{1}{n} \sum (f(x) - f^*(x;\mathbf{a})) \cdot (\frac{d}{d\mathbf{a}} f^*(x;\mathbf{a})) \\ \\
                      &= \frac{1}{n} \sum (f(x) - f^*(x;\mathbf{a})) \cdot \left[ \begin{array}{c} x^3 \\ x^2 \\ x \\ 1 \end{array} \right]
\end{align*}
$$

In [ ]:
def gradient(X, Y):
    
    err = Y - f_star(X)

    return np.array([
        np.mean(err * (X**3)),
        np.mean(err * (X**2)),
        np.mean(err * X),
        np.mean(err * 1)
    ])

We use this gardient along with a learning rate $\alpha$ in the **stochastic gradient descent** algorithm to *learn* the true constants (weights).

In [ ]:
X_train = X[:int(len(X)*0.95)]
Y_train = Y[:int(len(Y)*0.95)]
X_test = X[int(len(X)*0.95):]
Y_test = Y[int(len(Y)*0.95):]

lr = 0.000000001
epochs = 5000
train_loss = []
test_loss = []

for e in range(epochs):

    # compute the gradient
    grad = gradient(X_train, Y_train)
    
    # update the parameters
    a += lr * (-grad)

    
    if (e % 10 == 0):
        # compute the training and testing loss
        train_loss.append([e, Loss(X_train, Y_train)])
        test_loss.append([e, Loss(X_test, Y_test)])

    if (e % 10 == 0) or (e == epochs - 1):
        print(f"epoch: {e}, train loss: {train_loss[e//10][1]}, test loss: {test_loss[e//10][1]}")

In [ ]:
plt.plot(X, Y, 'b.', markersize=2, label='data')
plt.plot(X, f_star(X), 'r-', label='model')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('fitted cubic function')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
test_loss = np.array(test_loss)
train_loss = np.array(train_loss)
print(min(test_loss[:,1]), min(train_loss[:,1]))